In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:
def rmsle(actual_column, predicted_column):
    sum=0.0
    for x,y in zip(actual_column,predicted_column):
        if x<0 or y<0: #check for negative values. 
            continue
        p = np.log(y+1)
        r = np.log(x+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted_column))**0.5

In [7]:
data=pd.read_csv("..\\Train.csv")
data=data.set_index('ID')
print(data.head(3))

le_account_type=LabelEncoder()
data['account_type']=le_account_type.fit_transform(data['account_type'])

le_gender=LabelEncoder()
data['gender']=le_gender.fit_transform(data['gender'])

le_loan_enq=LabelEncoder()
data['loan_enq']=le_loan_enq.fit_transform(data['loan_enq'])

for col in data.columns:
    if data[col].skew()>0:
        data[col]=np.sqrt(data[col])
    else :
        data[col]=np.square(data[col])

cols=data.corr().cc_cons[abs(data.corr().cc_cons)>0.001].index

df2=data[cols]
print(df2.columns)

df2.investment_4=df2.investment_4.fillna(0)
print(df2.isna().sum())

X=df2.drop('cc_cons',1)
y=df2['cc_cons']
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.25, random_state=25)

      account_type gender  age  region_code  cc_cons_apr  dc_cons_apr  \
ID                                                                      
12554      current      M   35          708     24893.00        378.0   
17645      current      M   35          123     18941.62        966.0   
7604       current      M   55          802      5678.87       2724.0   

       cc_cons_may  dc_cons_may  cc_cons_jun  dc_cons_jun  ...  \
ID                                                         ...   
12554      10288.0     29664.00     16291.40      11432.0  ...   
17645      20672.0       287.00      4217.00        885.0  ...   
7604        1964.5      3933.11     23956.25       5168.0  ...   

       debit_count_may  max_credit_amount_may  debit_amount_jun  \
ID                                                                
12554               40                24563.0          30051.32   
17645               78                23948.0          42119.05   
7604                82             

C:\Users\AP hatela\Miniconda\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
# model fitting and predictions with rmsle score
lr=LinearRegression()
lr.fit(X_train, y_train)
train_pred=lr.predict(X_train)
val_pred=lr.predict(X_val)
train_rmsle=rmsle(train_pred, y_train)
val_rmsle=rmsle(val_pred, y_val)
print(train_rmsle, val_rmsle)

0.8574030124772111 0.866626882637528


In [10]:
# preparation of X_test

test=pd.read_csv('..\\Test.csv')
test=test.set_index('ID')
print(test.head(3))

test['account_type']=le_account_type.transform(test['account_type'])

test['gender']=le_gender.transform(test['gender'])

test['loan_enq']=le_loan_enq.transform(test['loan_enq'])

for col in test.columns:
    if test[col].skew()>0:
        test[col]=np.sqrt(test[col])
    else:
        test[col]=np.square(test[col])

X_test=test[cols]

X_test['investment_4']=X_test.fillna(0)
X_test.isna().sum()

      account_type gender  age  region_code  cc_cons_apr  dc_cons_apr  \
ID                                                                      
17591      current      M   37          575      2795.45       1290.0   
13541      current      M   33          394     29405.13       1640.0   
13431      current      M   53          324      5985.20       6189.0   

       cc_cons_may  dc_cons_may  cc_cons_jun  dc_cons_jun  ...  \
ID                                                         ...   
17591      6141.05       676.50       9146.1     15479.00  ...   
13541      1670.00      2463.92      10947.5       956.00  ...   
13431      2696.09       766.00        716.0      4133.22  ...   

       credit_count_may  debit_count_may  max_credit_amount_may  \
ID                                                                
17591              24.0             20.0                 9750.0   
13541              43.0             40.0                16967.0   
13431              42.0            

C:\Users\AP hatela\Miniconda\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


KeyError: "['cc_cons'] not in index"

In [8]:
y_pred=lr.predict(X_test)
y_pred**2

array([30570.02431664, 32371.69032086, 32095.68161125, ...,
       26083.93918803, 27024.10852273, 32240.96191847])

In [10]:
# storing the predicted results

X_test['cc_cons']=y_pred**2

result4=pd.DataFrame(X_test['cc_cons'])

print(result4.head())

result4.to_csv('submission4.csv',index=True)

            cc_cons
ID                 
17591  30570.024317
13541  32371.690321
13431  32095.681611
8687   38768.014784
14727  32103.588421
